# Text Sentiment Analysis
In this notebook I look at customer reviews for the top banks in Kenya. Is it enough to use social media comments to make an informed decision a given bank?
I will be using data from Twitter in this analysis

In [19]:
# Get Access tokens
%reload_ext dotenv
%dotenv

import os
import tweepy
from tweepy import OAuthHandler

# get access
# keys are stored in a '.env' file
consumer_key = os.getenv('CONSUMER_KEY')
consumer_secret= os.getenv('CONSUMER_SECRET')
access_token =  os.getenv('ACCESS_TOKEN')
access_secret =  os.getenv('ACCESS_SECRET')

# authorize access
oath = OAuthHandler(consumer_key,consumer_secret)
oath.set_access_token(access_token,access_secret)

#create api
twitter_api = tweepy.API(oath)


In [21]:
# examine top 10 tweets on timeline
for tweet in tweepy.Cursor(twitter_api.home_timeline).items(10):
    print(tweet.text)


Online Ad selection Using the Thompson Sampling method https://t.co/rMtIP6RIyF
Learn about tech innovators in 3D graphics, satellite imagery, collaborative editing, and other fields. https://t.co/S4B4OBSmA7
“It’s unfortunate, quite frankly, that as a nation we have now made schools a political football. That is going to… https://t.co/nQp0rKVp15
Pfizer CEO Albert Bourla said that the clinical trials should reveal by October whether the company’s vaccine again… https://t.co/lffzeWsOJN
#DevNews is a brand new podcast — &amp; #DevDiscuss is also fresh on the scene! Because of that, we'd LOVE reviews from… https://t.co/Bmn757Gshb
Ugh, fine. I 'get' Twitch now. All it took was 'Ghost of Tsushima' and a pandemic. https://t.co/1JiNPo2ENt https://t.co/SwqlA3j5Ng
RT @RuthGotian: It happened.

I received the reviews for a manuscript I submitted. I was told to read &amp; cite the work of Ruth Gotian.

I am…
👍 https://t.co/8Hh761hD1n
Well said https://t.co/Wjhvb6wysC
Thanks to the internet, gay tee

In [102]:
# twitter_api.search('bank')
# search banking tweets
# banks_list = ["KCB Bank","Equity Bank","I&M Bank","Cooperative Bank","Family Bank",
#               "DTB Bank","Barclays Bank","Standard Chartered Bank","National Bank","NIC Bank"]
search_list  = ['KCB Bank']
start_date = "2020-01-01"
ke_banking_tweets = []
for tweet in tweepy.Cursor(twitter_api.search,q=search_list,since=start_date).items(20):
    ke_banking_tweets.append(tweet.text)


In [103]:
len(ke_banking_tweets)

0

In [95]:
for i in range(10,20):
    print(ke_banking_tweets[i])

RT @Dadsaysjokes: A priest, an imam, and a rabbit walk into a blood bank.

The rabbit says,"I think I might be a type O."
RT @mmpadellan: BREAKING: Deutsche Bank COMPLIED with a subpoena from the Manhattan DA for trump's financial records.

TIK. TOK.
https://t.…
RT @lyndsay_barna: .@MTFootball18 @cade_clancy says "They've been working the safety plan the best we can and I think we deserve the best s…
@thejohngoodwin I cant wait for "Facebook Bank", that way they just get ALL of our personal info
RT @washingtonpost: Justice Department accuses Ukrainian oligarch of stealing billions from bank he once owned and laundering it in the U.S…
RT @FreddieBailey96: For millions of young people, there never was a bank of mum and dad.
RT @outroride: PLEASE DONATE TO LEBANON !! These are some trusted places to donate to:

Lebanese red cross 
🔗https://t.co/W1gj30Z2bq

Leban…
RT @mmpadellan: BREAKING: Deutsche Bank COMPLIED with a subpoena from the Manhattan DA for trump's financial records.

TIK. TOK